Read in stroop data, starting from "before" run, compile all data into one for each subject (each sheet). Remove unnecessary outputs such as labels other than the column names. Count and remove missing inputs and errors (double entries). Calculate overall time mean and median. Calculate standard deviation and 4 standard deviations to find outliers (4 SD from mean). Calculate each of the stroop conditions means, medians, SDs. 

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

#read in excel spreadsheet
path = '/Users/kjung6/Eva/Stroop/hivpd23/copy/HIVPD23_SMtS_behav.xlsx'
sheets = pd.read_excel(path, sheet_name=None, header=None)
output_path = '/Users/kjung6/Eva/Stroop/filtered_sheets.xlsx'

In [2]:
#define the values we want in column E (index 4)
valid_values = ['conM', 'conMNM', 'conNM', 'incM', 'incMNM', 'incNM']#, 'EventTag']

filtered_sheets = {}

#loop over each sheet and filter data based on valid values in column E
for sheet_name, sheet_data in sheets.items():
    #filter the sheet based on the values in column E (index 4)
    filtered_data = sheet_data[sheet_data.iloc[:, 4].isin(valid_values)]
    filtered_data = filtered_data.iloc[:, :8]
    filtered_data.columns = ['Trial', 'RunLabel', 'Condition', 'TrialStart', 'EventTag', 'Time', 'keys', 'match_status']
    #, 'sequence', 'mouse_down']
    filtered_sheets[sheet_name] = filtered_data

for sheet_name, sheet_data in filtered_sheets.items():
    #drop duplicates to get rid of extra colnames
    sheet_data = sheet_data.drop_duplicates()
    #update the filtered_sheets with the cleaned data
    filtered_sheets[sheet_name] = sheet_data

# #add new column to each sheet (pandas df) in dictionary (filtered_sheets) for match or nonmatch status called match_status
# for sheet_name, sheet_data in filtered_sheets.items():
#    sheet_data['match_status'] = ''

#count # of each unique val in 'keys' column of each df
for sheet_name, sheet_data in filtered_sheets.items():
    counts = sheet_data['keys'].value_counts()
    print(counts)



keys
[1]          114
[2]          110
[1][5]_UP      1
Name: count, dtype: int64
keys
[1]    99
[2]    88
Name: count, dtype: int64
keys
[1]    119
[2]     89
[4]     25
Name: count, dtype: int64
keys
[1]          122
[2]           84
[4]           24
[2][5]_UP      1
Name: count, dtype: int64
keys
[1]    117
[2]    113
Name: count, dtype: int64
keys
[1]          94
[2]          90
[3]          24
[4]          24
[1][5]_UP     1
Name: count, dtype: int64
keys
[1]          120
[2]          113
[2][5]_UP      1
Name: count, dtype: int64
keys
[2]          125
[1]          121
[2][5]_UP      2
[2][5]         1
[1][5]         1
Name: count, dtype: int64
keys
[2]    124
[1]    113
Name: count, dtype: int64
keys
[2]    99
[1]    84
[4]    24
Name: count, dtype: int64
keys
[2]    110
[1]     95
[3]     27
Name: count, dtype: int64
keys
[2]          108
[1]           94
[3]           23
[2][5]_UP      1
Name: count, dtype: int64
keys
[2]    111
[1]     95
[4]     25
Name: count, dtype: int64
k

In [28]:
#print(filtered_sheets['HP23-00057'].head())
subj_57 = filtered_sheets['HP23-00057']
subj_57_sorted = subj_57.sort_values(by = 'EventTag', ascending = True)
#print(subj_57_sorted)
subj_57_condition_stats = pd.DataFrame()
subj_57_condition_stats['metrics'] = ['mean', 'median', 'stdev']


#OVERALL

match = '[1]'
nonmatch = '[2]'
def map_condition(value):
    if 'conM' in value or 'incM' in value:
        return '[1]'
    elif 'conNM' in value or 'incNM' in value:
        return '[2]'
    else:
        return value  # keep the original value if no match

#apply the function to the 'Condition' column
subj_57_sorted['Condition_bin'] = subj_57_sorted['Condition'].apply(map_condition)
#print(subj_57_sorted)

#count the number of differences
num_differences = (subj_57_sorted['Condition_bin']  != subj_57_sorted['keys']).sum()
#print(num_differences)

#if different, delete row (by subsetting only rows that match)
subj_57_sorted = subj_57_sorted[subj_57_sorted['Condition_bin']  == subj_57_sorted['keys']]

#count num missing
missing_count = subj_57_sorted['keys'].isna().sum()
subj_57_sorted = subj_57_sorted[subj_57_sorted['keys'].isna() != True]

#mean overall rt
overall_mean = subj_57_sorted['Time'].mean()

#median overall rt
overall_median = subj_57_sorted['Time'].median()

#overall sd and 4x
overall_sd = subj_57_sorted['Time'].std()
sd4x = overall_sd * 4

#outlier cutoff
outlier_cut = overall_mean + sd4x
#print(outlier_cut)

subj_57_overall_stats = pd.DataFrame()
subj_57_overall_stats['metrics'] = ['misses', 'errors', 'overallRTmean', 'overallRTmedian', 'stdev', '4STDEV', 'outliers']
#subj_57_overall_stats['values'] = [''] * len(subj_57_overall) 
subj_57_overall_stats['values'] = [missing_count, num_differences, overall_mean, overall_median, overall_sd, sd4x, outlier_cut]

#subset outliers
outlier_df = subj_57_sorted[subj_57_sorted['Time'] >= outlier_cut]
subj_57_sorted = subj_57_sorted[subj_57_sorted['Time'] <= outlier_cut]


#conditional statistics

#conM-RR 
#conM_df = subj_57_sorted[subj_57_sorted['EventTag'].str.contains('conM', case=False, na=False)]
conM_df = subj_57_sorted[subj_57_sorted['EventTag'] == 'conM']
subj_57_condition_stats['conM-RR'] = [conM_df['Time'].mean(), conM_df['Time'].median(), conM_df['Time'].std()]

#incM-RR 
#incM_df = subj_57_sorted[subj_57_sorted['EventTag'].str.contains('incM', case=False, na=False)]
incM_df = subj_57_sorted[subj_57_sorted['EventTag'] == 'incM']
subj_57_condition_stats['incM-RR'] = [incM_df['Time'].mean(), incM_df['Time'].median(), incM_df['Time'].std()]

#conM-RS 
#df that is all rows other than the conM rr ones
#non_conM_df = subj_57_sorted[~subj_57_sorted['EventTag'].str.contains('conM', case=False, na=False)]
non_conM_df = subj_57_sorted[subj_57_sorted['EventTag'] != 'conM']
#df that is all conM Conditions that are in non_conM_df aka all the rs conM's
conM_rs_df = non_conM_df[non_conM_df['Condition'].str.contains('conM', case=False, na=False)]
subj_57_condition_stats['conM-RS'] = [conM_rs_df['Time'].mean(), conM_rs_df['Time'].median(), conM_rs_df['Time'].std()]

#incM-RS
#df that is all rows other than incM rr ones
non_incM_df = subj_57_sorted[subj_57_sorted['EventTag'] != 'incM']
#df that is all incM Conditions that are in non_conM_df aka all the rs incM's 
incM_rs_df = non_incM_df[non_incM_df['Condition'].str.contains('incM', case=False, na=False)]
subj_57_condition_stats['incM-RS'] = [incM_rs_df['Time'].mean(), incM_rs_df['Time'].median(), incM_rs_df['Time'].std()]

#conNM-RR
conNM_df = subj_57_sorted[subj_57_sorted['EventTag'] == 'conNM']
subj_57_condition_stats['conNM-RR'] = [conNM_df['Time'].mean(), conNM_df['Time'].median(), conNM_df['Time'].std()]

#incNM-RR
incNM_df = subj_57_sorted[subj_57_sorted['EventTag'] == 'incNM']
subj_57_condition_stats['incNM-RR'] = [incNM_df['Time'].mean(), incNM_df['Time'].median(), incNM_df['Time'].std()]

#conNM-RS
#df that is all rows other than the conNM rr ones
non_conNM_df = subj_57_sorted[subj_57_sorted['EventTag'] != 'conNM']
#df that is all conNM Conditions that are in non_conNM_df aka all the rs conNM's
conNM_rs_df = non_conNM_df[non_conNM_df['Condition'].str.contains('conNM', case=False, na=False)]
subj_57_condition_stats['conNM-RS'] = [conNM_rs_df['Time'].mean(), conNM_rs_df['Time'].median(), conNM_rs_df['Time'].std()]

#incNM-RS
#df that is all rows other than the conNM rr ones
non_incNM_df = subj_57_sorted[subj_57_sorted['EventTag'] != 'incNM']
#df that is all incNM Conditions that are in non_incNM_df aka all the rs incNM's
incNM_rs_df = non_incNM_df[non_incNM_df['Condition'].str.contains('incNM', case=False, na=False)]
subj_57_condition_stats['incNM-RS'] = [incNM_rs_df['Time'].mean(), incNM_rs_df['Time'].median(), incNM_rs_df['Time'].std()]

#con-RR
#con_df = subj_57_sorted[subj_57_sorted['EventTag'].str.contains('conM', case=False, na=False)]
RR_df = subj_57_sorted[~subj_57_sorted['EventTag'].str.contains('MNM', case=False, na=False)]
con_RR_df = RR_df[RR_df['Condition'].str.contains("con", case=False, na=False)]
subj_57_condition_stats['con-RR'] = [con_RR_df['Time'].mean(), con_RR_df['Time'].median(), con_RR_df['Time'].std()]

#inc-RR
inc_RR_df = RR_df[RR_df['Condition'].str.contains("inc", case=False, na=False)]
subj_57_condition_stats['inc-RR'] = [inc_RR_df['Time'].mean(), inc_RR_df['Time'].median(), inc_RR_df['Time'].std()]

#con-RS
RS_df = subj_57_sorted[subj_57_sorted['EventTag'].str.contains('MNM', case=False, na=False)]
con_RS_df = RS_df[RS_df['Condition'].str.contains("con", case=False, na=False)]
subj_57_condition_stats['con-RS'] = [con_RS_df['Time'].mean(), con_RS_df['Time'].median(), con_RS_df['Time'].std()]

#inc-RS
inc_RS_df = RS_df[RS_df['Condition'].str.contains("inc", case=False, na=False)]
subj_57_condition_stats['inc-RS'] = [inc_RS_df['Time'].mean(), inc_RS_df['Time'].median(), inc_RS_df['Time'].std()]



In [6]:
#under 150 ms is "anticipation" and needs to be removed.
print(subj_57_sorted["Time"].min())
#none below 150ms

#print(subj_57_stats['conM-RR'].min())

557
77.66032730282309


In [19]:

#write out to excel
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for sheet_name, filtered_data in filtered_sheets.items():
        filtered_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
for sheet_name, sheet_data in filtered_sheets.items():
    print(sheet_name)